In [1]:
from dotenv import load_dotenv
import os
from openai import OpenAI

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")


if openai_api_key:
    print(f"OpenAI API Key exists and begins:{openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins: {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins:sk-proj-
Anthropic API Key exists and begins: sk-ant-


In [3]:
client = OpenAI()
MODEL = "gpt-4o-mini"
chat_completion = client.chat.completions.create(
    messages=[{"role": "user", "content": "What is an AI Agent?"}],
    model=MODEL,
    temperature=0,
)

print(chat_completion.choices[0].message.content)

An AI agent is a system or entity that uses artificial intelligence techniques to perceive its environment, make decisions, and take actions to achieve specific goals. AI agents can operate autonomously or semi-autonomously and can be found in various applications, ranging from simple rule-based systems to complex machine learning models.

Key characteristics of AI agents include:

1. **Perception**: AI agents can gather information from their environment through sensors or data inputs. This could involve processing visual data, audio signals, or other forms of input.

2. **Decision-Making**: Based on the information they perceive, AI agents can analyze the data and make decisions. This can involve reasoning, planning, and learning from past experiences.

3. **Action**: After making decisions, AI agents can take actions to interact with their environment. This could involve physical actions (like a robot moving) or digital actions (like a software agent executing commands).

4. **Auton

In [4]:
class Agent:
    def __init__(self, client, model=MODEL, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
            result = self.execute()
            self.messages.append({"role": "assistant", "content": result})
            return result
        else:
            print("NO MESSAGE")

    def execute(self):
        completion = client.chat.completions.create(model=MODEL, messages=self.messages)
        return completion.choices[0].message.content

In [5]:
system_prompt = """


You run in a loop of THOUGHT, ACTION, PAUSE, OBSERVATION.

When you have a THOUGHT you return the THOUGHT and ACTION

At the end of the loop you output an Answer

Use THOUGHT to describe your THOUGHTs about the question you have been asked.

Use ACTION to run one of the ACTIONs available to you - then return PAUSE.

OBSERVATION will be the result of running those ACTIONs.


Your available ACTIONs are:


## calculate_total

if amount = 200

then calculate_total(amount)


This returns amount * 1.2


Runs the calculate_total function and returns a JSON FORMAT output as follows:

{"result": 240, "fn": "calculate_total", "next": "PAUSE"}


## get_product_price

This uses the get_product_price function and passes in the value of the product

e.g. get_product_price('bike')


This uses the get_product_price with a product = 'bike', finds the price of the bike and then returns a JSON FORMAT output as follows:

{"result": 200, "fn": "get_product_price", "next": "PAUSE"}


Example session:



Question: What is the cost of a bike??

THOUGHT: I need to find the cost of a bike|ACTION|get_product_price|bike


You will be called again with this:


OBSERVATION|200


THOUGHT: I need to calculate the total including the VAT|ACTION|calculate_total|200



You will be called again with this: 


OBSERVATION|240


If you have the answer, output it as the ANSWER.


ANSWER|The price of the bike including VAT is 240


Now it's your turn:

""".strip()


def calculate_total(amount) -> float:

    return int(amount * 1.2)


def get_product_price(product):

    if product == "bike":

        return 100

    if product == "tv":

        return 200

    if product == "laptop":

        return 300

    return None

In [6]:
agent = Agent(client=client, system=system_prompt)

In [7]:
result = agent("What is cost of a laptop including the VAT?")
result = agent("What is cost of a bike including the VAT?")
result = agent("What is cost of a tv including the VAT?")


print(result)

THOUGHT: I need to find the cost of a TV|ACTION|get_product_price|tv


In [8]:
next = result.split("|")
print(next)
next_function = next[2]
next_arg = next[3]
print(next_function, next_arg)

['THOUGHT: I need to find the cost of a TV', 'ACTION', 'get_product_price', 'tv']
get_product_price tv


In [9]:
print(next_function, "|", next_arg)
if next_function == "get_product_price":
    result = get_product_price(next_arg)
    print(result)

get_product_price | tv
200


In [10]:
next_prompt = f"Observation|{result}"
next_prompt

'Observation|200'

In [11]:
result = agent(next_prompt)
print(result)

THOUGHT: I need to calculate the total for the TV including the VAT|ACTION|calculate_total|200


In [12]:
next = result.split("|")
print(next)
next_function = next[2]
next_arg = int(next[3])
print(f"FUNCTION - {next_function} |  ARGUMENT - {next_arg}")

['THOUGHT: I need to calculate the total for the TV including the VAT', 'ACTION', 'calculate_total', '200']
FUNCTION - calculate_total |  ARGUMENT - 200


In [13]:
print(next_function, "|", next_arg)
if next_function == "calculate_total":
    result = calculate_total(next_arg)
    print(result)

calculate_total | 200
240


In [14]:
next_prompt = f"OBSERVATION|{result}"
next_prompt

'OBSERVATION|240'

In [15]:
result = agent(next_prompt)
print(result)

ANSWER|The price of the TV including VAT is 240
